## 신경망 학습

In [1]:
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist

In [2]:
def cross_entropy_error(y ,t ):
    delta = 1e-7
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t* np.log(y + delta))/ batch_size

* 신경망에서의 기울기

In [3]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

* Softmax

In [4]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

* Sigmoid

In [5]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

* 2층 신경망 구현하기

In [25]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        # 예측 구하기 (가중치의 합 -> 시그모이드 통과 -> 가중치의 합 -> 소프트맥스 통과)
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)

        a2 = np.dot(z1, W2) + b2              
        y = softmax(a2)
        
        return y
        
    def loss(self, x, t):
        # 손실(비용) 함수의 결과 구하기
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        # 그레디언트(편미분의 벡터) 구하기
        loss_W = lambda W : self.loss(x, t)
        
        grads= {}
        
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    #def gradient()
        
    

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=20, output_size=10)

# 하이퍼 파라미터
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.2


train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    grad = network.numerical_gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] = network.params[key] - (learning_rate * grad[key])

        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    print(i, loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

0 2.297595224052484
train acc, test acc | 0.09736666666666667, 0.0982
1 2.297152499681115
2 2.294134761713491
3 2.294104239919292
4 2.2905969955837504
5 2.2924529556816022
6 2.2936783616551857
7 2.285898596029104
8 2.286369954592652
9 2.2958526652610853
10 2.300489496424446
11 2.2877069063662447
12 2.295929126378159
13 2.290554105703375
14 2.291644337939617
15 2.29856084415194
16 2.2971796222944594
17 2.2840706489088016
18 2.2801234132860055
19 2.278070115911828
20 2.2936323227887687
21 2.28991461516979
22 2.3005078221427526
23 2.294327635053684
24 2.292474461352863
25 2.2977041819565036
26 2.302767655937253
27 2.3026087395131323
28 2.297853776662176
29 2.292423084273714
30 2.290453900721938
31 2.290524948817208
32 2.301862302745714
33 2.2981454000847505
34 2.2805969806475272
35 2.2981402913834525
36 2.289791262462929
37 2.29477281365874
38 2.2770370554142305
39 2.2974110460033983
40 2.28915066298501
41 2.2740544520007098
42 2.285625922961909
43 2.285175734535119
44 2.297733480165006
4

In [24]:
60000/100

600.0